# Subspace Digital Twin, Initial Conditions Run

*Shawn Anderson, January 2024*

In this notebook, we examine medianl behavior over the first 90 days.

## Part 1. Dependences & Set-up

Autoreload modules while developing.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

import hvplot.pandas
hvplot.extension('bokeh')

from bokeh.models import HoverTool
import holoviews as hv

from bokeh.palettes import Turbo256, Category20

from subspace_model.util import g


2024-03-25 11:36:28 - subspace-digital-twin - INFO
------------subspace-digital-twin------------


## Part 2. Load Simulation Data

Load the simulation results data.

In [2]:
import glob
import os

def load_latest_simulation(simulation_name):
    files = glob.glob(f"../data/simulations/{simulation_name}-*.pkl.gz")
    latest_file = max(files, key=os.path.getctime)

    print(f"Loading Latest File: {latest_file}")
    df = pd.read_pickle(latest_file)
    df = df.drop(['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'], axis=1)
    return df

In [3]:
sim_df = load_latest_simulation("reference_subsidy_sweep")
sim_df

Loading Latest File: ../data/simulations/reference_subsidy_sweep-2024-03-25_11-35-50.pkl.gz


,days_passed,blocks_passed,circulating_supply,user_supply,earned_supply,issued_supply,earned_minus_burned_supply,total_supply,sum_of_stocks,storage_fee_per_rewards,block_utilization,avg_blockspace_usage,dsf_relative_disbursal_per_day,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,burnt_balance,nominator_pool_shares,operator_pool_shares,block_reward,blockchain_history_size,total_space_pledged,allocated_tokens,buffer_size,reference_subsidy,average_priority_fee,average_compute_weight_per_tx,average_transaction_size,transaction_count,average_compute_weight_per_bundle,average_bundle_size,bundle_count,compute_fee_multiplier,compute_fee_volume,free_space,extrinsic_length_in_bytes,storage_fee_in_credits_per_bytes,storage_fee_volume,priority_fee_volume,consensus_extrinsic_fee_volume,rewards_to_nominators,max_normal_weight,max_bundle_weight,target_block_fullness,adjustment_variable,storage_fees_to_farmers,storage_fees_to_fund,target_block_delta,targeted_adjustment_parameter,tx_compute_weight,run,average_compute_weight_per_budle,label,environmental_label,max_credit_supply
0,0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,1.134600e+09,1.865400e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0,0,0.0,0.0,0.000000,0.0,0.0,256,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000e+00,1,NaN,constant-single-component,standard,3000000000
15,1,14400.0,14.982834,14.982834,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.000000,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.000000,0.000000,0.749142,14.233692,0.000000,0.000000,0.000000e+00,0.000000,0.000000,14.982834,2038498852864,103497886643200,0.0,60045568.0,13.698630,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,0.000000e+00,3.145888e+10,1.019216e+12,0.059296,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000
30,2,28800.0,24.541582,24.579039,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.039033,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.003746,0.003746,1.575430,22.958661,0.000000,0.037457,0.000000e+00,0.000000,0.000000,9.596205,4076997705728,205422829286400,0.0,120091136.0,13.698630,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,1.000000e-18,3.145888e+10,1.019216e+12,0.059296,0.374571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337114,0.037457,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000
45,3,43200.0,34.058941,34.175245,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.082086,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.011400,0.011585,2.435337,31.600619,0.000075,0.116229,0.000000e+00,0.000037,0.000037,9.596205,6115764994048,307361193702400,0.0,45918976.0,13.698630,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,1.000000e-18,3.145888e+10,1.019216e+12,0.059296,0.787715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.708943,0.078771,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000
60,4,57600.0,43.533150,43.771450,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.126891,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.022898,0.023646,3.308516,40.178090,0.000305,0.237995,7.116846e-09,0.000153,0.000151,9.596205,8154263846912,409286136345600,0.0,105964544.0,13.698630,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,1.000000e-18,3.145888e+10,1.019216e+12,0.059296,1.217668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.095901,0.121767,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16187,357,5140800.0,6609.407013,12246.332428,0.0,1.865412e+09,0.0

In [4]:
# sim_df = pd.read_pickle(
#     "../data/simulations/reference_subsidy_sweep-2024-01-30_11-07-21.pkl.gz"
# ).drop(['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'], axis=1)

In [5]:
sim_df.head(5)

,days_passed,blocks_passed,circulating_supply,user_supply,earned_supply,issued_supply,earned_minus_burned_supply,total_supply,sum_of_stocks,storage_fee_per_rewards,block_utilization,avg_blockspace_usage,dsf_relative_disbursal_per_day,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,burnt_balance,nominator_pool_shares,operator_pool_shares,block_reward,blockchain_history_size,total_space_pledged,allocated_tokens,buffer_size,reference_subsidy,average_priority_fee,average_compute_weight_per_tx,average_transaction_size,transaction_count,average_compute_weight_per_bundle,average_bundle_size,bundle_count,compute_fee_multiplier,compute_fee_volume,free_space,extrinsic_length_in_bytes,storage_fee_in_credits_per_bytes,storage_fee_volume,priority_fee_volume,consensus_extrinsic_fee_volume,rewards_to_nominators,max_normal_weight,max_bundle_weight,target_block_fullness,adjustment_variable,storage_fees_to_farmers,storage_fees_to_fund,target_block_delta,targeted_adjustment_parameter,tx_compute_weight,run,average_compute_weight_per_budle,label,environmental_label,max_credit_supply
0,0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,1.134600e+09,1.865400e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0,0,0.0,0.0,0.00000,0.0,0.0,256,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000e+00,1,NaN,constant-single-component,standard,3000000000
15,1,14400.0,14.982834,14.982834,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.000000,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.000000,0.000000,0.749142,14.233692,0.000000,0.000000,0.000000e+00,0.000000,0.000000,14.982834,2038498852864,103497886643200,0.0,60045568.0,13.69863,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,0.000000e+00,3.145888e+10,1.019216e+12,0.059296,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000
30,2,28800.0,24.541582,24.579039,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.039033,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.003746,0.003746,1.575430,22.958661,0.000000,0.037457,0.000000e+00,0.000000,0.000000,9.596205,4076997705728,205422829286400,0.0,120091136.0,13.69863,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,1.000000e-18,3.145888e+10,1.019216e+12,0.059296,0.374571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337114,0.037457,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000
45,3,43200.0,34.058941,34.175245,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.082086,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.011400,0.011585,2.435337,31.600619,0.000075,0.116229,0.000000e+00,0.000037,0.000037,9.596205,6115764994048,307361193702400,0.0,45918976.0,13.69863,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,1.000000e-18,3.145888e+10,1.019216e+12,0.059296,0.787715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.708943,0.078771,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000
60,4,57600.0,43.533150,43.771450,0.0,1.865400e+09,0.0,1.865400e+09,3.000000e+09,0.126891,0.5,0.0,0.0,1.134600e+09,1.865400e+09,0.022898,0.023646,3.308516,40.178090,0.000305,0.237995,7.116846e-09,0.000153,0.000151,9.596205,8154263846912,409286136345600,0.0,105964544.0,13.69863,0.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.0,1.000000e-18,3.145888e+10,1.019216e+12,0.059296,1.217668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.095901,0.121767,-0.5,1.0,2.388787e+17,1,1.000000e+10,constant-single-component,standard,3000000000


Simulation Runs.

In [6]:
sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days').head()

,run,label,environmental_label,Days
0,1,constant-single-component,standard,362
1,2,hybrid-single-component,standard,362
2,3,hybrid-two-components,standard,362


### Coloring Metrics
Use a constant mapping from columns to colors

In [7]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

In [8]:
def snake_to_title(s):
    """Utility function used for printing chart titles and labels as Title Case.
    Example:
    snake_to_caps('snake_case')
    >>> 'Snake Case'
    """
    
    return ' '.join(word.capitalize() for word in s.split('_'))

def fan_chart_quantile_median(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of median for a series."""

    # min, max, median
    fan_df = df.groupby('days_passed')[column].agg(['min', 'max', 'median'])

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Fan Chart', ylabel=f'{column}_min_max_median')

    # Median curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    median_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Median', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return median_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='min', y2='max', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * median_chart
    return chart


def fan_chart_quantile(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of quantile for a series."""

    # 25%, 50%, 75%
    fan_df = df.groupby('days_passed')[column].quantile([0.25, 0.5, 0.75]).unstack().rename(columns={0.50:'median', 0.25:'0.25',0.75:'0.75'})

    # return fan_df

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Quantile Fan Chart', ylabel=f'{column}_quantile')

    # Quantile curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    quatile_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Quantile', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return quatile_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='0.25', y2='0.75', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * quatile_chart
    return chart


### KPIs

In [9]:
sim_df['issuance'] = sim_df['block_reward'] + sim_df['reference_subsidy']

sim_df['fees'] = sim_df['compute_fee_volume'] + sim_df['storage_fee_volume']

fees_and_issuance = ['compute_fee_volume','storage_fee_volume', 'fees', 'block_reward', 'reference_subsidy', 'issuance']

In [10]:
# Compute Fees and Storage Fees

# The dynamics of storage fees vs issuance. Who will dominate at the beginning, storage fees or issues rewards? Note that this is a metric.
# Another metrics of interest, general revenue per timestep, farmers, proposers, voters, and data blocks
# revenue = proposer_reward + storage_fees. For data blocks and voters you only have rewards not fees. Farmers is the sum of those three.
# The above topics are what has been discussed and therefor are higher priority than the stocks. 

In [11]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

### Balances and Supplies

In [12]:
system_balances = ['other_issuance_balance', 'reward_issuance_balance']
agent_balances = [
    'farmers_balance',
    'operators_balance',
    'nominators_balance',
    'holders_balance',
]
agent_pool_balances = ['staking_pool_balance']
protocol_treasury_balances = ['fund_balance']
other_balances = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances + agent_balances + agent_pool_balances + protocol_treasury_balances) )
supply_columns = list({c for c in sim_df.columns if 'supply' in c} - {'max_credit_supply', 'issued_supply', 'total_supply'})
balance_columns = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances))

In [13]:
system_balances

['other_issuance_balance', 'reward_issuance_balance']

In [14]:
agent_balances

['farmers_balance',
 'operators_balance',
 'nominators_balance',
 'holders_balance']

In [15]:
agent_pool_balances

['staking_pool_balance']

In [16]:
protocol_treasury_balances

['fund_balance']

In [17]:
other_balances

['burnt_balance']

In [18]:
supply_columns

['earned_minus_burned_supply',
 'earned_supply',
 'user_supply',
 'circulating_supply']

In [19]:
balance_columns

['fund_balance',
 'nominators_balance',
 'holders_balance',
 'staking_pool_balance',
 'farmers_balance',
 'operators_balance',
 'burnt_balance']

In [20]:
balance_columns

['fund_balance',
 'nominators_balance',
 'holders_balance',
 'staking_pool_balance',
 'farmers_balance',
 'operators_balance',
 'burnt_balance']

In [21]:
supply_columns

['earned_minus_burned_supply',
 'earned_supply',
 'user_supply',
 'circulating_supply']

In [22]:
fees_and_issuance

['compute_fee_volume',
 'storage_fee_volume',
 'fees',
 'block_reward',
 'reference_subsidy',
 'issuance']

In [23]:
# balance_columns = fees_and_issuance
# balance_columns = supply_columns

In [24]:
box_df = sim_df.set_index(['days_passed', 'label'])[balance_columns]
box_df

,,fund_balance,nominators_balance,holders_balance,staking_pool_balance,farmers_balance,operators_balance,burnt_balance
days_passed,label,,,,,,,
0,constant-single-component,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
1,constant-single-component,0.000000,0.000000,0.749142,0.000000,14.233692,0.000000,0.000000e+00
2,constant-single-component,0.037457,0.003746,1.575430,0.000000,22.958661,0.003746,0.000000e+00
3,constant-single-component,0.116229,0.011585,2.435337,0.000075,31.600619,0.011400,0.000000e+00
4,constant-single-component,0.237995,0.023646,3.308516,0.000305,40.178090,0.022898,7.116846e-09
...,...,...,...,...,...,...,...,...
357,hybrid-two-components,5256.848907,218.135139,596.548473,383.834114,5746.451201,48.272200,3.757606e+00
358,hybrid-two-components,5286.676331,218.936549,597.748096,386.459804,5757.911811,48.382765,3.794070e+00
359,hybrid-two-components,5316.563736,219.735943,598.941420,389.094351,5769.312228,48.492751,3.830784e+00


In [25]:
describe_df = box_df.describe().drop('count')
describe_df

,fund_balance,nominators_balance,holders_balance,staking_pool_balance,farmers_balance,operators_balance,burnt_balance
mean,1323.983998,63.782330,206.973275,87.448754,2000.005769,16.001444,0.693743
std,1303.356614,54.636294,146.413037,96.058946,1409.098836,11.963096,0.920633
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,263.353878,19.426983,99.710099,9.687382,962.875555,7.134553,0.022782
50%,923.019022,49.829434,155.225772,52.552890,1506.586264,11.883209,0.266389
75%,1957.192833,100.134878,302.716400,128.503436,2916.051790,24.054531,1.034741
max,5376.517231,221.328639,601.309298,394.389910,5791.933753,48.710992,3.904963


In [26]:
describe_labels_df = box_df.groupby('label').apply(lambda label: label.describe().drop('count'))
describe_labels_df

fund_balance  nominators_balance  \
label                                                              
constant-single-component mean    645.989670           30.397926   
                          std     525.588974           19.852442   
                          min       0.000000            0.000000   
                          25%     155.556125           11.712457   
                          50%     545.607522           31.392046   
                          75%    1081.476825           48.633723   
                          max    1708.006609           61.374695   
hybrid-single-component   mean   1550.375209           72.955023   
                          std    1261.413538           47.645860   
                          min       0.000000            0.000000   
                          25%     373.334699           28.109897   
                          50%    1309.458054           75.340910   
                          75%    2595.544379          116.720936   
                          max    4099.215862          147.299268   
hybrid-two-components     mean   1775.587115           87.994040   
                          std    1587.880308           67.215004   
                          min       0.000000            0.000000   
                          25%     373.334699           28.109897   
                          50%    1309.458054           75.340910   
                          75%    2949.305635          143.568195   
                          max    5376.517231          221.328639   

                                holders_balance  staking_pool_balance  \
label                                                                   
constant-single-component mean        94.768705             43.443357   
                          std         41.278719             41.152717   
                          min          0.000000              0.000000   
                          25%         64.818147              5.673767   
                          50%        105.310988             30.781136   
                          75%        130.486203             75.279516   
                          max        146.139115            134.544871   
hybrid-single-component   mean       227.444893            104.264057   
                          std         99.068925             98.766522   
                          min          0.000000              0.000000   
                          25%        155.563554             13.617041   
                          50%        252.746372             73.874725   
                          75%        313.166887            180.670839   
                          max        350.733877            322.907691   
hybrid-two-components     mean       298.706226            114.638849   
                          std        177.277406            115.403581   
                          min          0.000000              0.000000   
                          25%        155.563554             13.617041   
                          50%        252.746372             73.874725   
                          75%        465.237641            193.321211   
                          max        601.309298            394.389910   

                                farmers_balance  operators_balance  \
label                                                                
constant-single-component mean       915.132239           7.402772   
                          std        394.564699           3.676716   
                          min          0.000000           0.000000   
                          25%        629.088970           4.581634   
                          50%       1015.948656           8.308235   
                          75%       1256.461029          10.628635   
                          max       1405.998905          12.071382   
hybrid-single-component   mean      2196.317373          17.766653   
                          std        946.955278           8.824119   
                 

In [27]:
describe_difference_df = pd.DataFrame(describe_labels_df.values - pd.concat([describe_df for i in range(sim_df['label'].nunique())]).values, columns=describe_labels_df.columns, index=describe_labels_df.index)
df = describe_difference_df

def log_scale(val, max_abs_log):
    """ Apply logarithmic scaling to a value. """
    if val == 0:
        return 0
    else:
        return np.sign(val) * np.log(abs(val) + 1) / max_abs_log

def color_scale(val):
    max_abs_val = df.abs().max().max()
    max_abs_log = np.log(max_abs_val + 1)

    scaled_val = log_scale(val, max_abs_log)

    if scaled_val < 0:
        intensity = int(255 * (1 + scaled_val))  # More negative, more red
        return f'background-color: rgb(255, {intensity}, {intensity})'
    elif scaled_val > 0:
        intensity = int(255 * (1 - scaled_val))  # More positive, more green
        return f'background-color: rgb({intensity}, 255, {intensity})'
    else:
        return 'background-color: rgb(255, 255, 255)'

header_styles = [{
    'selector': f'th.col_heading.level0.col{i}',
    'props': [('background-color', column_colors.get(col))]
} for i, col in enumerate(df.columns)]

df.columns.name = 'balance'

describe_difference_df_styled = df.style.map(color_scale).set_table_styles(header_styles)
describe_difference_df_styled

In [28]:
box_df_melted = box_df.reset_index().drop('days_passed',axis=1).melt(id_vars=['label'])

violin_list = [label.hvplot.violin(y='value', by='variable', c='variable', legend='top_left', width=1200, height=500, title=f'SSC Balances {name}', cmap=column_colors, ylim=(0,box_df.max().max()*0.75)) for name, label in box_df_melted.groupby('label')]

# Combine plots into a single column layout
layout = hv.Layout(violin_list).cols(1)

layout

:Layout
   .Violin.I   :Violin   [variable]   (value)
   .Violin.II  :Violin   [variable]   (value)
   .Violin.III :Violin   [variable]   (value)

In [29]:
[label for name, label in box_df.reset_index().groupby('label')][0]

,days_passed,label,fund_balance,nominators_balance,holders_balance,staking_pool_balance,farmers_balance,operators_balance,burnt_balance
0,0,constant-single-component,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
1,1,constant-single-component,0.000000,0.000000,0.749142,0.000000,14.233692,0.000000,0.000000e+00
2,2,constant-single-component,0.037457,0.003746,1.575430,0.000000,22.958661,0.003746,0.000000e+00
3,3,constant-single-component,0.116229,0.011585,2.435337,0.000075,31.600619,0.011400,0.000000e+00
4,4,constant-single-component,0.237995,0.023646,3.308516,0.000305,40.178090,0.022898,7.116846e-09
...,...,...,...,...,...,...,...,...,...
357,357,constant-single-component,1678.847067,60.901907,145.591427,131.673181,1400.766716,12.020901,1.430791e+00
358,358,constant-single-component,1686.126638,61.020852,145.729432,132.389242,1402.085111,12.033621,1.443300e+00
359,359,constant-single-component,1693.413110,61.139297,145.866713,133.106547,1403.396583,12.046274,1.455877e+00
360,360,constant-single-component,1700.706445,61.257244,146.003273,133.825093,1404.701169,12.058861,1.468522e+00


In [30]:
line_list = [hv.Overlay([fan_chart_quantile(label, column) for column in label.columns if column not in ['label', 'days_passed']]).opts(title=f'SSC Balances {name}', legend_opts={'location':'top_left'}) for name, label in box_df.reset_index().groupby('label')]
layout = hv.Layout(line_list).cols(1)
layout

:Layout
   .Overlay.I   :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
   .Overlay.II  :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
   .Overlay.III :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)

In [31]:
violin_list = [variable.hvplot.violin(y='value', by='label', color=column_colors[name], width=1200, height=500, title=f'SSC Balances {name}', ylim=(0,variable.max()['value'].max()), ylabel=name) for name, variable in box_df_melted.groupby('variable')]

layout = hv.Layout(violin_list).cols(1).opts(shared_axes=False)

layout

:Layout
   .Violin.I   :Violin   [label]   (value)
   .Violin.II  :Violin   [label]   (value)
   .Violin.III :Violin   [label]   (value)
   .Violin.IV  :Violin   [label]   (value)
   .Violin.V   :Violin   [label]   (value)
   .Violin.VI  :Violin   [label]   (value)
   .Violin.VII :Violin   [label]   (value)

In [32]:
line_list = [variable.hvplot.line(x='days_passed', by='label', y='value', title=name, legend='top_left', line_width=3).opts(legend_opts={'background_fill_color': column_colors[name], 'background_fill_alpha': 0.2}) for name, variable in box_df.reset_index().melt(id_vars=['label', 'days_passed']).groupby('variable')]

layout = hv.Layout(line_list).cols(2).opts(shared_axes=False)
layout

:Layout
   .NdOverlay.I   :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.II  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.III :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.IV  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.V   :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.VI  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.VII :NdOverlay   [label]
      :Curve   [days_passed]   (value)

Definition (per timestep) storage_fees_per_rewards = state['storage_fee_volume'] / state['block_reward']

We are interested in having the (95%, 50%, 5%) quantile distribution over that metric when taking windows of 1 week, 4 weeks and 12 weeks.

In [33]:
opts = dict(width=800, height=400)
charts = []
for weeks in [1,4,12]:
    sim_df['storage_fee_per_rewards_q1'] = sim_df['storage_fee_per_rewards'].rolling(7*weeks).quantile(0.05)
    sim_df['storage_fee_per_rewards_q2'] = sim_df['storage_fee_per_rewards'].rolling(7*weeks).quantile(0.50)
    sim_df['storage_fee_per_rewards_q3'] = sim_df['storage_fee_per_rewards'].rolling(7*weeks).quantile(0.95)
    
    fan = sim_df.hvplot.area(x='days_passed', y='storage_fee_per_rewards_q1', y2='storage_fee_per_rewards_q3', by='label', stacked=False, hover=False, legend='bottom_right')
    
    median = sim_df.hvplot.line(x='days_passed', y='storage_fee_per_rewards_q2', by='label', alpha=0.8, line_width=4, ylim=(0,None), title=f'Storage Fee Per Rewards 95% Quantile Rolling {weeks} weeks.', legend='bottom_right')
    chart = fan * median
    charts.append(chart.opts(**opts))

In [34]:
hv.Layout(charts).cols(3)

:Layout
   .Overlay.I   :Overlay
      .NdOverlay.I  :NdOverlay   [label]
         :Area   [days_passed]   (storage_fee_per_rewards_q1,storage_fee_per_rewards_q3)
      .NdOverlay.II :NdOverlay   [label]
         :Curve   [days_passed]   (storage_fee_per_rewards_q2)
   .Overlay.II  :Overlay
      .NdOverlay.I  :NdOverlay   [label]
         :Area   [days_passed]   (storage_fee_per_rewards_q1,storage_fee_per_rewards_q3)
      .NdOverlay.II :NdOverlay   [label]
         :Curve   [days_passed]   (storage_fee_per_rewards_q2)
   .Overlay.III :Overlay
      .NdOverlay.I  :NdOverlay   [label]
         :Area   [days_passed]   (storage_fee_per_rewards_q1,storage_fee_per_rewards_q3)
      .NdOverlay.II :NdOverlay   [label]
         :Curve   [days_passed]   (storage_fee_per_rewards_q2)